# Forward Propagation

A neural network consists of a collection of neurons, arranged into layers. It's a type of machine learning algorithm, so its objective is to take input data, then output transformed data.  We usually imagine neural networks as looking like this:

<img src="neural_network.jpg" alt="A neural network" style="width: 400px;"/>

We usually think of the data as a matrix, where each row is a different data point. The row itself is a (potentially long) vector, where each entry is a different *feature* of the data. The output of the network is also a matrix, where each row is the "answer" from the input data.

Neural networks are often used for classification, meaning the output will be a vector of (numbers which are nearly) zeros and ones, where each column represents being in a particular class. We might also use a neural network for regression, so the output could be a vector of numbers which are potentially unbounded, and have some kind of meaning.

Neural networks are organized into *layers*, so that we transform the data by each layer successively, until after we apply the final layer and get the answer.  Neuron layers transform each row "in-place;" that is, after applying each layer of the network, the data has the same number of rows, and each row is a transformed copy of the original row.

We think of each column of that transformed data as the output of a particular neuron, so if the data has (say) 500 columns after applying layer 3, then we think of layer 3 as having 500 neurons.

We think of the neurons themselves as taking a $1\times n$ vector of input $\mathbf{x}$, applying an affine transformation $\mathbf{x}\mathbf{w}+b$ to get a real number (so $\mathbf{w}$ is $n\times 1$ and $b$ is $1\times 1$), then applying some nonlinear *activation function* f, so that the output of the neuron is $f(\mathbf{x}\mathbf{w}+b)$.

The specific activation function that we choose will be discussed below, but there are plenty of options.  The point is that we use a nonlinear function, so that after lots of layers, the resulting function can be very complicated.  If we don't apply an activation function, or if it is linear, then no matter how many layers we go through, we only get a linear function at the end.

If a layer has $k$ neurons, then we will have a weight *matrix* which is $n\times k$, so each *column* is a neuron.  It will also have a bias *vector* which is $1\times k$, so each *column* (of the vector) is a bias number.  So when we apply the transformation to the $m\times n$ data $\mathbf{x}$, we get $\mathbf{x}\mathbf{w}$, which is $m\times k$, then need to add the bias vector $b$ to each row, which requires us to scale up the vector to a matrix.

Note that to scale up the bias vector $b$, we need it to be an $m\times k$ matrix, where each row is the original bias vector $b$.  The scaled version of $b$ is just $ub$, where $u$ is an $m\times 1$ matrix of all ones.

The process of taking input, applying the layers of neurons successively, and ending up with the output is called **forward propagation**.

## Architectures

At each level, we need to choose several things. Neural networks can be really complicated, and if you implement all the things researchers regularly work on, it can be quite a lot. But the things we need to decide right now are:
1. The number of layers, and the dimensions of each layer.
3. The activation function at each layer.
2. The weights and bias of each neuron in each layer.

We consider (1) and (2) the **architecture** of the network; (3) will usually be randomized at first and then optimized using training data.  We'll talk about (2) in the next segment.

As for (1), we certainly need at least one layer.  It's possible to use only one layer, but this isn't very interesting, so we'll almost always use at least two (and often more).  The first layer takes as input the actual input data; for this reason we call it the **input layer**.  Its input size must be equal to the size of the input, so that's fixed by the shape of the data.

Similarly, the final layer needs to give the actual output of the network, so it's called the **output layer**. The output dimension of this layer needs to be exactly what is required for the output, so that is also fixed by the data.

Finally, if the output of layer $i$ has $k$ columns, then the input of layer $i+1$ has $k$ columns, since it takes as input exactly the output of the preceding layer.

Therefore, we are choosing a sequence of positive integers $k_0,\ldots,k_l$, where $k_1$ is the size (that is, number of columns of the output) of the input layer, $k_2$ is the size of the next layer, and so on.  The number $l$ is the number of hidden layers of the network; if $l=0$ there are no hidden layers, so the input layer feeds directly into the output layer.  If $l=-1$ (that is, we choose nothing at all) then the input layer is the output layer.

We will write our code in such a way that if it receives the empty sequence, then the network generated is a single layer which is both input and output.

### Notation

Let's look at our image again:

<img src="neural_network.jpg" alt="A neural network" style="width: 400px;"/>

Networks are often drawn this way.  In this picture, the circles represent the numerical outputs of each layer.  The *input layer*, as drawn on the picture, is literally the input of the network.  The *output layer*, similarly, is the literal output of the network.  We are more concerned with the neurons than the sequential outputs, so we have referred to the neurons with their weights and biases as the layers. In this picture, the layers (as we have described them above) are more like the arrows, taking one layer to another.

We need to fix some notation, which we do now.

Dimensions:
1. $n$ is the dimensionality of the input (in columns).
2. $k$ is the dimensionality of the output (in columns).
3. $l$ is the number of layers, where $0$ means the input layer feeds directly into the output layer.
4. $k_i$ is the dimensionality of the output (in columns) of layer $i$, so $k_{-1}=n$ and $k_l=k$.

Parameters of Each Layer:
1. $W_i$ is the matrix of weights at layer $i$, so $W_i$ is a $k_{i-1}\times k_i$ matrix.
2. $b_i$ is the vector of biases at layer $i$, so $b_i$ is a $k_i\times 1$ matrix.
3. $f_i$ is the activation function at layer $i$.

Inputs and Outputs:
1. $x_i$ is the input to layer $i$, so $x_i$ is a $1\times k_{i-1}$ (row) matrix. Note that $x_0$ is the actual input to the network.
2. $z_i$ is the *non-activated* output of layer $i$, so $z_i=x_iw_i+b_i$.
3. $y_i$ is the output of layer $i$, so $y_i=f_i(z_i)$.  Note that $y_i=x_{i+1}$, assuming there is a next layer to take input.  Note that $y_l$ is the output of the matrix and $y_{-1}$ is the input to the matrix.

There will be a lot more as we go, but this is enough to get started.

## Code

Here is our forward propagation code:

In [1]:
def forward_prop(weights, biases, activations, input_data):
    l = len(weights)
    
    x = [0] * l # input to level i
    z = [0] * l # un-activated output of level i
    y = [0] * l # activated output of level i
    
    x[0] = input_data
    
    for i in range(0, l):
        expanded_bias = np.ones((x[i].shape[0], 1)) * biases[i]
        z[i] = np.dot(x[i], weights[i]) + expanded_bias
        y[i] = activations[i](z[i])
        
        if i < l-1:
            x[i+1] = y[i]
    
    return x, z, y

It may not be immediately obvious why we need $x$, $z$, and $y$, but all of them will have their role in the training process.